# Neural Machine Translation(NMT) model

This project had the goal to review the Transformer architecture, delve into an NLP task and be inspired by existing implementation for future personal work.

# Transformers

The architecture is based on 2 parts:

- Encoder
- Decoder

Given the sentence: “my name is Simone” I have the following:

- After the encoder part, I get 4 vectors
- then these vectors are passed to the decoder.

---

To start predicting:

- We pass the START token
- and these 4 vectors

We get the first prediction (corrisponding to the first word)

Then, we pass the prediction to the decoder and we obtain the second.

We repeat this until we get an <end> tag. 

---


## Encoder architecture:

Given the inputs,  after positional encoding, we get vector of a fixed length (512 in the paper) (EMBEDDINGS)

---
We pass them to the MHA and get new context vectors:
- Intuition: These new vectors will be with higher qualitiy and containing much more context than the first ones due to the Multi-head attention part.

---

# Multi-head attention mechanism

## Attention mechanism (single head)

Each word is going to have 3 vectors (multiplying  it with three different weight matrices Wq, Wk and Wv which are continually updated during training):

- Query vector (is the representation for the word we want to calculate the attention)
- Key vector (representation of all the words)
- Value vector (is the actual representation of each word)
- The output is a new vector for each query

This is done in parallel thanks to how the attention works for each query.

---

##  Computing the new vector for each Query:

Multiplying the Query and Key gives us a score that tells us how much weight each Value Vj (and thus, its corresponding word) obtains in the self-attention vector.

How?

- Q1 * K1 = score1 —> softamax(score1 / rad(dk) ) * V1
- Q1 * K2 = score2  —> softamax(score2 / rad(dk) )  * V2
- ….
- Z1 = V1 + V2 + … + VK

---

# Multi-Head Attention

The process above is carried out multiple times with DIFFERENT WEIGHT MATRICES FOR EACH HEAD.

It means we end up with multiple vectors (called Heads). These heads are then concatenated and multiplied with a weight matrix Wo. This means that each head learns different information about a given sequence and that this knowledge is combined at the end.

---

# Decoder

---



## Masking

- At decoding time we need to use masking while training.

- Basically, the idea is instead of doing one prediction at the time we can pass all the actual predictions one time, but to do this we need to mask the future.

### Intuition

- We pass to the decoder all the outputs [start tag, output_1, output_2, … , output_K]

- After positional encoding, the output is passed to the Masked Multi-head attention layer.

- Here the idea is when decoding we want to attend only the previous words, not the future.

- Eg. when predicting the output 3, we want to attend only the outputs: start tag, output_1, output_2.

---

# Positional Encodings:

- Positional encoding describes the location or position of an entity in a sequence so that each position is assigned a unique representation.

- Transformers use a smart positional encoding scheme, where each position/index is mapped to a vector.

- The output of the positional encoding layer is a matrix, where each row of the matrix represents an encoded object of the sequence summed with its positional information

## A quick recap of sine/cosine functions:

- The function’s range is [-1,+1].
- The frequency of this waveform is the number of cycles completed in one second.
- The wavelength is the distance over which the waveform repeats itself.

 Suppose you have an input sequence of length L  and require the position of the  object K within this sequence. 

The positional encoding is given by sine and cosine functions of varying frequencies.

## Intuition:

- Given d that is the embedding size, we fill a new vector of size d.

- Where at each position i have either a sin() for the pair positions or a cos() for the odd positions

---

# Stacked Encoder/Decoder

Basically each encoder gives k outputs given k input but they are passed as the input to the encoder itself for 6 times before passing them to the decoder.

---

# Layer Normalization

- The activations of each layer will have a center of 0 and a sd of 1 in order to have a more stable training and fast convergence (this is different from Batch normalization)

## Batch norm against Layer Norm

- Basically, the normalization in Layer norm is done across the input and not across the batch;
- Therefore there is not need to use a running average at test time due to having everything we need;
- Yet, in Layer Norm there are two learnable parameters as in Batch Norm;

---

# Recap: 

## Encoder

1. Given a sentence, i need to create a vector for each word (the vector encodes better the meaning of each word)
    1. This process is done in parallel thanks to the multi-head attention layer
    2. Then at decoding time, each past context vectors are used for decoding each  new word 
2. The steps:
    1. fixing the sentence lenght using padding (Max sequence length)
    2. encoding each word as vector  ( each vector has dimension equals to Max sequence length)
    3. Embedding transform (to have denser vectors,  usually dimension now is 512) 
        1. the parameters here are learnt in the back propagation from the end of the decoder
    4. then positional encodings are added (generating a vector of dim 512 for each word)
        1. why? well the attention mechanism is treating the input as an unordered sequence so since we are talking about sentences and the order is important here we need to encode the position.
    5. now each vector is mapped to a query, key, value  vector (always a 512 vector) using 3 matrixes (all learnable parameters) 
    6. And they are passed to the multi-head attention
        1. For each head, computing the attention matrix (with dimension Max sequence lenght x Max sequence length)
        2. then passing it into a softmax function scaling with a constant
        3. we get the self-attention matrixes (every value is a prob)
        4. we multiply with the value vector to get the actual context vector
    7. we concatened all the context vectors obtained with the multi-heads (just adding)
    8. we pass toghether with a residual connection to the Norm layer (to have a stable training)
    9. then pass to a Feed Forward ( a sequence of FC layers, varying dim along the path)
    10. then again layer norm
    11. still each vecort has 512 as dimension;
    12. we repeat all these operatio for 12 times and then we get the final context vectors;

---

## Decoder:

- everyword has a dim in our case 512
- the staring point is the <start> tag
- the ending point is the <end> tag
- the padding is used when i want to fix the max sequence length

As before first step is to add position encodings

and then for each vector: query, key, value vector each with 512 as dim

then masked self attention for each head

1. computing the not normalized attention matrix  for each head
2. then we scale
3. but we need to mask! since we don’t want to cheat at inference time!
    1. this means that when i’m doing inference i don’t have the future words and so i can’t know the corrisponding keys values. then i create a mask where the next steps are put to -inf and therefore when computing the attention score after the softmax i have zero for the future attention scores and so they model puts 0 attention to the next words, treating them as they were a padding! (to avodi Data leakage)

then we concatene the context vectors for the multi head!

Then adding the residual connection to avoid the vanishing gradient problem due to the backpropagation in many layers!

Then layer normalization!

Then multi head cross attention!

how:

1. taking the query vector from the decoder
2. and the key and value vector from the encoder
3. How?
    1. Basically, each context vector is transformed in a key and value vector
    2. therefore if we have 3 context vectors we have 3 keys and 3 values
4. Then we compute the attention for the query (eg. the translated previous query) with the keys of the context vectors
    1. here we don’t need the masked attention since we can use all the inputs passed to the encoder part.
    2. then we concatenate the heads as before
5. Layer norm
6. Feed forward and Layer norm with residual connection to get the starting dimension of the input
7. Repeat these steps K times to learn the complexity of the language
8. Pass the result to a Feed Forward that is mappnig to a vocabulary and then a softmax for the next word prob
    1. in the training each row is prob distro for the next word, they are used for computing the loss ignoring the padding; then backpropagating the loss in paralled taking advantages of batches;
    2. at test time, we only have the start token and all the pad tokens, when we reach the final step of the decoder we only consider the first row and then pass the prediction to the decoder and so on!

---



To understand better:

[https://jalammar.github.io/illustrated-transformer/](https://jalammar.github.io/illustrated-transformer/)

---



# Input token pre-processing:

1. Define:
    - START_TOKEN
    - PADDING_TOKEN
    - END_TOKEN

- START_VOCABULARY
- END_VOCABULARY

Each is a sound in the alphabet!

1. Define:
    - Index_to_ENDVocabulary
    - ENDVocabulary_to_index
    - Index_to_STARTVocabulary
    - STARTVocabulary_to_index
    
2. Extract the sentences (eg. splitting from “\n”)

1. Create embedding for the characters instead the word
    1. but first check for valid sentences in terms of length and vocabulary

1. Create the dataset for the training 
    1. define a custom dataset 
2. Define a tokenize function (to transform from words to numbers)
    1. that is inserting the start, end , padding token when needed

1. Now transoform the sentences keeping in mind that only for one language we need the start and end token;
2. Keep in mind: we need also a padding mask for the backprogation (the loss cant’ be influenced by the pad)

#  IMPLEMENTATION:

FOLLOWING: https://www.scaler.com/topics/keras/neural-machine-translation-model-in-keras/

TO INSTALL TENSORFLOW_TEXT: https://medium.com/@lakshmi_priya_ramisetty/how-to-install-tensorflow-text-on-mac-m1-m2-0bacb6fe83d1

https://github.com/keras-team/keras-io/blob/master/examples/nlp/neural_machine_translation_with_keras_nlp.py